### Notebook for concept detection in neural network

In [1]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
board_name = "5x5"

agents_to_sample = [0, 150, 300, 450, 600]

full_model_path = "../models/saved_sessions/board_size_5/"
board_size = 5
dims = (4, 5, 5)

CONCEPT_FUNC = concepts.concept_area_advantage

CONCEPT_NAME = concepts.concept_area_advantage.__name__


2023-10-17 14:16:46.586117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [3]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2

    game_over = False

    while not game_over:
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)
        
        action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

In [4]:
positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 100 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")
while len(positive_cases) < CASES_TO_SAMPLE:
    pos, neg = play_match(agents, board_size, CONCEPT_FUNC)
    positive_cases.extend(pos)
    negative_cases.extend(neg)
    positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

Positive cases: 100%|██████████| 100/100 [00:16<00:00,  3.53it/s]

In [5]:
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (100, 4, 5, 5)
Negative cases:  (100, 4, 5, 5)


In [17]:
epoch_to_look_at = [0, 150, 300, 450, 600]

for epoch in epoch_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])
    np.random.shuffle(shuffled_indices)
    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]
    POSITIONS_TO_CONSIDER = 40 #40000
    VALIDATION_POSITIONS = 10 #10000

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs


    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print("Performing regression for layer {}".format(i))
        # points = np.concatenate([points, actual_outputs], axis=1)
        score = concepts.perform_regression(
            points[:POSITIONS_TO_CONSIDER], 
            all_labels[:POSITIONS_TO_CONSIDER], 
            points[POSITIONS_TO_CONSIDER:], 
            all_labels[POSITIONS_TO_CONSIDER:], 
            True
        )
        concept_presence_per_layer.append(score)

        print("The presence of {} in resblock {} is {}".format(CONCEPT_NAME, i, score))
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Save concept presences in json file
    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, model_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, model_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, model_name, CONCEPT_NAME, epoch_to_look_at), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, model_name, CONCEPT_NAME, epoch_to_look_at, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 7/7 [00:00<00:00, 258.60it/s]

Performing regression for layer 0
Epoch 1/50


2/2 [==============================] - 1s 142ms/step - loss: 1.0139 - val_loss: 1.0414
Epoch 2/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0104 - val_loss: 1.0383
Epoch 3/50
2/2 [==============================] - 0s 23ms/step - loss: 1.0074 - val_loss: 1.0350
Epoch 4/50
2/2 [==============================] - 0s 27ms/step - loss: 1.0045 - val_loss: 1.0315
Epoch 5/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0016 - val_loss: 1.0280
Epoch 6/50
2/2 [==============================] - 0s 26ms/step - loss: 0.9987 - val_loss: 1.0248
Epoch 7/50
2/2 [==============================] - 0s 25ms/step - loss: 0.9957 - val_loss: 1.0216
Epoch 8/50
2/2 [==============================] - 0s 25ms/step - loss: 0.9928 - val_loss: 1.0185
Epoch 9/50
2/2 [==============================] - 0s 25ms/step - loss: 0.9900 - val_loss: 1.0154
Epoch 10/50
2/2 [==============================] - 0s 24ms/step - loss: 0.9872 - val_loss: 1.0124
Epoch 11/50
2/2 [======================

Getting resblock outputs: 100%|██████████| 7/7 [00:00<00:00, 257.25it/s]

Performing regression for layer 0
Epoch 1/50


...

Performing regression for layer 0
Epoch 1/50


2/2 [==============================] - 0s 110ms/step - loss: 1.0632 - val_loss: 1.0294
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0534 - val_loss: 1.0226
Epoch 3/50
2/2 [==============================] - 0s 25ms/step - loss: 1.0448 - val_loss: 1.0162
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 1.0356 - val_loss: 1.0100
Epoch 5/50
2/2 [==============================] - 0s 25ms/step - loss: 1.0264 - val_loss: 1.0039
Epoch 6/50
2/2 [==============================] - 0s 26ms/step - loss: 1.0184 - val_loss: 0.9980
Epoch 7/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0099 - val_loss: 0.9922
Epoch 8/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0032 - val_loss: 0.9866
Epoch 9/50
2/2 [==============================] - 0s 25ms/step - loss: 0.9950 - val_loss: 0.9813
Epoch 10/50
2/2 [==============================] - 0s 24ms/step - loss: 0.9877 - val_loss: 0.9762
Epoch 11/50
2/2 [======================

Getting resblock outputs: 100%|██████████| 7/7 [00:00<00:00, 261.78it/s]

Performing regression for layer 0
Epoch 1/50


2/2 [==============================] - 0s 124ms/step - loss: 1.0211 - val_loss: 1.0142
Epoch 2/50
2/2 [==============================] - 0s 28ms/step - loss: 1.0149 - val_loss: 1.0094
Epoch 3/50
2/2 [==============================] - 0s 28ms/step - loss: 1.0096 - val_loss: 1.0043
Epoch 4/50
2/2 [==============================] - 0s 27ms/step - loss: 1.0042 - val_loss: 0.9990
Epoch 5/50
2/2 [==============================] - 0s 26ms/step - loss: 0.9984 - val_loss: 0.9938
Epoch 6/50
2/2 [==============================] - 0s 26ms/step - loss: 0.9931 - val_loss: 0.9888
Epoch 7/50
2/2 [==============================] - 0s 26ms/step - loss: 0.9871 - val_loss: 0.9839
Epoch 8/50
2/2 [==============================] - 0s 26ms/step - loss: 0.9822 - val_loss: 0.9790
Epoch 9/50
2/2 [==============================] - 0s 24ms/step - loss: 0.9766 - val_loss: 0.9742
Epoch 10/50
2/2 [==============================] - 0s 24ms/step - loss: 0.9713 - val_loss: 0.9694
Epoch 11/50
2/2 [======================